# Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice.


## Q1. Running Elastic
Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

In [1]:
!curl localhost:9200

{
  "name" : "39b86c2f4153",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "EE0stasrSJGF8qiCLycrmQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


What's the version.build_hash value?
## Getting the data
Now let's get the FAQ data. You can run this snippet:

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Note that you need to have the requests library:

In [3]:
!pip install requests

## Q2. Indexing the data
Index the data in the same way as was shown in the course videos. Make the ``course`` field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

Which function do you use for adding your data to elastic?

``pip install elasticsearch``

* insert
*  index
*  put
*  add


In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '39b86c2f4153', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'EE0stasrSJGF8qiCLycrmQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/jCUhR720Q_W2yyMKlgg_bQ] already exists')

Now we're ready to index all the documents:

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

## Q3. Searching
Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only ``question`` and ``text`` fields and give ``question`` a boost of 4, and use ``"type": "best_fields"``.

What's the score for the top ranking result?

    94.05
    84.05
    74.05
    64.05

Look at the ``_score`` field.

In [ ]:
user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [ ]:
response = es.search(index=index_name, body=search_query)
response

## Q4. Filtering

Now let's only limit the questions to ``machine-learning-zoomcamp``.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [ ]:
def retrieve_documents(query, index_name="course-questions", max_results=3):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [ ]:
# 1
user_question = "How do I execute a command in a running docker container?"

response = retrieve_documents(user_question)

for doc in response:
    print("doc:  ", doc.keys())
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

## Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks ``(\n\n)``

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [ ]:
context_docs = retrieve_documents(user_question)

context_result = ""

for doc in context_docs:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()
print(context)

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context}
""".strip()

In [ ]:
def build_context(documents):
    context_result = ""
    
    for doc in documents:
        doc_str = context_template.format(**doc)
        context_result += ("\n\n" + doc_str)
    
    return context_result.strip()


def build_prompt(user_question, documents):
    context = build_context(documents)
    prompt = prompt_template.format(
        user_question=user_question,
        context=context
    )
    return prompt


In [ ]:
user_question = "How do I execute a command in a running docker container?"


In [ ]:
prompt = build_prompt(user_question, context_docs)
prompt

In [ ]:
len(prompt)

What's the length of the resulting prompt? (use the len function)

*    962
*    1462
*    1962
*    2462


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses ``tiktoken`` for tokenization:

In [ ]:
!pip install tiktoken --q

**Let's calculate the number of tokens in our query:**

In [ ]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
encoding.encode(user_question), len(encoding.encode(prompt))

Use the encode function. How many tokens does our prompt have?

*    122
*    222
*    322
*    422



Note: to decode back a token into a word, you can use the ``decode_single_token_bytes`` function:

In [ ]:
encoding.decode_single_token_bytes(63842)

## Bonus: generating the answer (ungraded)
Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices here

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
  
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.

## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1
* It's possible that your answers won't match exactly. If it's the case, select the closest one.
